In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

%cd /content/drive/MyDrive/Github/
%cd greenhouse_text_status/
%pwd

/content/drive/MyDrive/Github
/content/drive/MyDrive/Github/greenhouse_text_status


'/content/drive/MyDrive/Github/greenhouse_text_status'

In [ ]:

from Load_Settings_File import LoadSettings
from pathlib import Path

In [ ]:

setting_file_path = Path("/content/drive/MyDrive/Github/settings.json")
#setting_file_path = Path("settings.json")
test_load = LoadSettings(file_path=setting_file_path)

the_dict = test_load.LoadSettingFile()

print(the_dict['email_address'])

In [ ]:

import imaplib
import email
from email.header import decode_header
import webbrowser
import os

# account credentials
username = the_dict['email_address']
password = the_dict['app_password']

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [ ]:

# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL("imap.gmail.com")
# authenticate
imap.login(username, password)

In [ ]:

status, messages = imap.select("INBOX")

#replace email_address with the string that the user is looking for....
#will add email contracts to the settings file... this list will be those that should be looked for
typ, msg_ids = imap.search(None, '(FROM "email_address")')
# number of top emails to fetch
N = 3
# total number of emails
messages = int(messages[0])

In [ ]:

print(messages)
print(typ)
print(msg_ids)
print(type(msg_ids))

for iv in msg_ids:
  print(iv)
  print(type(iv))

hg = msg_ids[0]

ff = hg.decode()

print(type(ff))
print(ff)


num_lst = ff.split(sep=" ")
print(num_lst)

469
OK
[b'456 457 458 469']
<class 'list'>
b'456 457 458 469'
<class 'bytes'>
<class 'str'>
456 457 458 469
['456', '457', '458', '469']


In [ ]:

#for i in range(messages, messages-N, -1):
for i in num_lst:
    i = int(i)
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            #subject, encoding = decode_header(msg["Subject"])[0]
            #if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                #subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            #print("Subject:", subject)
            print("Subject: empty")
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(f'This is the content_type: {content_type}')
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = msg.get_payload(decode=True).decode()
                if content_type == "text/plain":
                    # print only text email parts
                    print("text/plain")
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in browser
                folder_name = clean(subject)
                if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    os.mkdir(folder_name)
                filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                # write the file
                open(filepath, "w").write(body)
                # open in the default browser
                webbrowser.open(filepath)
            print("="*100)
# close the connection and logout
imap.close()
imap.logout()

In [ ]:
imap.close()
imap.logout()

('BYE', [b'LOGOUT Requested'])